In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import sent_tokenize
pd.set_option('max_colwidth', None)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [11]:
labeled_data = pd.read_csv('generated_snippets.csv').drop(columns=['Unnamed: 0'])
labeled_data.guidance = labeled_data.guidance.astype(int)
labeled_data = labeled_data.dropna().reset_index()
labeled_data['snippets']=labeled_data['snippets'].str.replace('\\n\\n','',regex=True)
labeled_data.head()
dataset = labeled_data

In [12]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenized_data = tokenizer(dataset['snippets'].values.tolist(), return_tensors="np", padding=True)['input_ids']


In [4]:
tokenized_data

array([[ 101, 2057, 3424, ...,    0,    0,    0],
       [ 101, 2057, 3613, ...,    0,    0,    0],
       [ 101, 2057, 3305, ...,    0,    0,    0],
       ...,
       [ 101, 2117, 1010, ...,    0,    0,    0],
       [ 101, 2117, 1010, ...,    0,    0,    0],
       [ 101, 2117, 1010, ...,    0,    0,    0]])

In [5]:


#vectorizer = TfidfVectorizer (max_features = 750)
#tokenized_data = vectorizer.fit_transform(dataset['snippets']).toarray()

labels = np.array(dataset["guidance"])  # Label is already an array of 0 and 1

rf = RandomForestClassifier(n_estimators=100)
X = tokenized_data
y = labels
X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.2)
rf.fit(X_train,y_train)



RandomForestClassifier()

In [6]:
labeled_data

,index,snippets,guidance
0,0,We anticipate net charge-offs to remain within a range of 3.9% to 4.3% for next year.,1
1,1,"We continue to monitor macroeconomic trends and the outlook for our customers and our industry, and we may make further adjustments to our outlook if developments indicate a need for us to change course. We are well positioned to deliver strong results for our shareholders regardless of what comes next.",1
2,2,"We understand the responsibility we have to manage our credit risks carefully and responsibly, and we are committed to proactive measures to protect our customer base and our bottom line.",1
3,5,"Given these dynamics, we anticipate that net charge-offs could increase to between 4.0% and 4.4% over the course of 2023. This range is in line with our expectations for the macroeconomic environment, while preserving the balance between responsible lending and keeping the cost of funds competitive.",1
4,6,"Longer-term, we expect charge-off rates to normalize within a range between 2.5% and 3.5%. Depending on absolute levels of charge-offs, our business strategy could be impacted if we are unable to absorb higher-than-anticipated levels. We consider this risk as part of our overall risk management process and will actively monitor charge-off levels to ensure that our business continues to perform in an effective manner.",1
...,...,...,...
817,995,"Additionally, we grew deposits by 22%, driven largely by our robust mobile capabilities and digital offerings that appeal to new customers. Finally, beyond our strong financial performance, we also saw customer satisfaction with our services increase by 7%, a testament to the investments we’ve made in customer experience.",0
818,996,"Second, operating income grew by 17%, driven by strong volume growth, increased productivity and improved expense management. This is reflective of our ongoing commitment to drive long-term value and generate consistent results.Finally, our customer satisfaction ratings remain high, indicating that existing customers are satisfied with our service.Overall, these results make it clear that our efforts are translating into meaningful value for our shareholders and that we are well-positioned to continue delivering strong returns.",0
819,997,"Second, we grew new deposits by 24% and saw a 14% improvement in deposit mix shifting more towards higher-margin products. Third, we grew net revenue year-over-year by 12%. This was driven by strong growth in both net interest income and non-interest income.Overall, these strong results exceed our expectations, demonstrate the effectiveness of our strategy, and set us up to continue leading the industry in digital banking. We are more confident than ever about our ability to drive future growth and deliver future value to our customers, employees, and shareholders.",0
820,998,"Second, our deposit accounts increased by 15%, showcasing our ability to leverage our digital platform and attract both existing and new customers. Third, we increased our market share in the mortgage segment by an impressive 28%. This indicates our growing ability to execute in this highly competitive space and is driven by our continued investment in our mortgage platform, operations and customer experience.",0


In [7]:
predictions = rf.predict(X_test)

In [8]:
round(accuracy_score(y_test, predictions),4)

0.9091

In [17]:

filex = open('discover_4q_call.txt', 'r')
text = filex.read()
tokens = nltk.sent_tokenize(text)
target_data = {"snippets":tokens}
target_data = pd.DataFrame(target_data)
target_data.head()

,snippets
0,Good morning.
1,"My name is Todd, and I will be your conference operator today."
2,"At this time, I would like to welcome everyone to the Fourth Quarter and Full Year 2022 Discover Financial Services Earnings Conference Call."
3,All lines have been placed on mute to prevent any background noise.
4,After the speakers remarks there will be a question-and-answer session.


In [31]:
dataset = target_data
#vectorizer = TfidfVectorizer (max_features=750)

new_data_processed = tokenizer(dataset['snippets'].values.tolist(), return_tensors="np", padding=True)['input_ids']



In [34]:
len(new_data_processed[0])

79

In [28]:


# vectorizer.fit(dataset['snippets'])
# new_data_processed = vectorizer.transform(target_data['snippets']).toarray()
y_pred = rf.predict(new_data_processed)

ValueError: X has 79 features, but RandomForestClassifier is expecting 238 features as input.

In [15]:
round(accuracy_score(y_test, predictions),4)

0.9091

In [ ]:
target_data['class_prediction'] = y_pred
target_data.head()


In [ ]:
import seaborn as sns
sns.countplot(x=pd.DataFrame(y_pred)[0])
plt.xticks([0,1],['guidance','not guidance snippets'])
plt.xlabel('snippets')

In [ ]:
dataset = labeled_data
from sklearn.linear_model import LogisticRegression

labels = np.array(dataset["guidance"])  # Label is already an array of 0 and 1

X = tokenized_data
y = labels
X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.2)
rf = LogisticRegression(random_state=0).fit(X_train, y_train)


In [ ]:
round(f1_score(y_test,predictions),4)

In [ ]:
dataset = target_data

y_pred = rf.predict_proba(new_data_processed)
target_data['probability'] = pd.DataFrame(y_pred)[[0]]
target_data.sort_values('probability',ascending=False)